In [1]:
ls

chuvash-wiki-parallel.zip  jupyter_1372519.log  jupyter_1387756.log
chv_rus_100K_Wiki_model/   jupyter_1386459.log  jupyter_1387985.log
data1/                     jupyter_1387267.log  jupyterhub_slurm/
EXP_ONLY_CORPUS/           jupyter_1387713.log  subword-nmt/
jupyter_1372508.log        jupyter_1387755.log  Wiki_model.ipynb


In [2]:
import torch
print(torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_name(i))

/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1
Tesla V100-SXM2-32GB


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import load_dataset

In [31]:
train_path = 'data1/wiki_train_all.csv'
test_path = 'data1/corpus_test.csv'

df_test = pd.read_csv(test_path)
df_train = load_dataset('csv', data_files=train_path).shuffle(seed=42)

split_dataset = df_train['train'].train_test_split(test_size=0.1)

Found cached dataset csv (/home/kvzhirnov/.cache/huggingface/datasets/csv/default-a12f4f926b2ce341/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 1/1 [00:00<00:00, 75.54it/s]
Loading cached shuffled indices for dataset at /home/kvzhirnov/.cache/huggingface/datasets/csv/default-a12f4f926b2ce341/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-788cf248a8434616.arrow


In [32]:
!mkdir 'data1/data'
!mkdir 'data1/data/train'
!mkdir 'data1/data/dev'
!mkdir 'data1/data/test'

In [33]:
def writelines(list, filename):
  with open(filename, 'w', encoding='utf-8') as file:
    for item in list:
      file.write('%s\n' % item)

chv_train = [l['chv'] for l in split_dataset['train']]
rus_train = [l['rus'] for l in split_dataset['train']]

chv_dev = [l['chv'] for l in split_dataset['test']]
rus_dev = [l['rus'] for l in split_dataset['test']]

chv_test = df_test['chv'].tolist()
rus_test = df_test['rus'].tolist()

writelines(chv_train, 'data1/data/train/train.chv.txt')
writelines(rus_train, 'data1/data/train/train.rus.txt')

writelines(chv_dev, 'data1/data/dev/dev.chv.txt')
writelines(rus_dev, 'data1/data/dev/dev.rus.txt')

writelines(chv_test, 'data1/data/test/test.chv.txt')
writelines(rus_test, 'data1/data/test/test.rus.txt')

In [34]:
file_name = 'data1/data/dev/dev.chv.txt'
number_of_rows = 5  # The number of rows you want to read

with open(file_name, 'r') as file:
    for i in range(number_of_rows):
        line = file.readline()
        print(line.strip())

Тĕн йăлине тытакан латышсем — ытларах лютерансемпе католицисем, çаплах православисем те пур.
"Пулăшу йытни", "Восстание" ( 1877 ), "Туссем", "Тĕнчене" т.ыт.
Кластерла аркану хыççăн 28Mg нукличĕ пулать (аркану пулаяслăхе 1,4(3) %, ытти данныйсемпе 3,9(10) %):
Патшалăх шыв реестрĕнчи шыв объекчĕн кочĕ — 03050100112103000060665
Альберт Эйнштейн — паллӑ физик.


In [35]:
file_name = 'data1/data/dev/dev.rus.txt'
number_of_rows = 5  # The number of rows you want to read

with open(file_name, 'r') as file:
    for i in range(number_of_rows):
        line = file.readline()
        print(line.strip())

Верующие латыши — преимущественно протестанты и католики, есть также православные.
Он протестовал против национального гнёта («Восстание 1877 г.»), мечтал о справедливом обществе, строе («Друзьям», «Миру»).
Кластерный распад с образованием нуклида 28Mg (вероятность распада 1,4(3) %, по другим данным 3,9(10) %):
Код водного объекта в государственном водном реестре — 03050100112103000060665
Из уроженцев Ульма наиболее известен Эйнштейн.


In [11]:
src = 'chv'
tgt = 'rus'
name = '100K_Wiki'

In [38]:
!python subword-nmt/subword_nmt/learn_joint_bpe_and_vocab.py --input data1/data/train/train.{src}.txt data1/data/train/train.{tgt}.txt  -s 10000 -o data1/data/bpe.codes  --write-vocabulary data1/data/bpe.vocab.{src} data1/data/bpe.vocab.{tgt}

100%|####################################| 10000/10000 [01:27<00:00, 114.38it/s]


In [39]:
!python subword-nmt/subword_nmt/apply_bpe.py -c data1/data/bpe.codes --vocabulary data1/data/bpe.vocab.{src} --vocabulary-threshold 50 < data1/data/train/train.{src}.txt > data1/data/train/train.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c data1/data/bpe.codes --vocabulary data1/data/bpe.vocab.{tgt} --vocabulary-threshold 50 < data1/data/train/train.{tgt}.txt > data1/data/train/train.{tgt}.bpe

!python subword-nmt/subword_nmt/apply_bpe.py -c data1/data/bpe.codes --vocabulary data1/data/bpe.vocab.{src} --vocabulary-threshold 50 < data1/data/dev/dev.{src}.txt > data1/data/dev/dev.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c data1/data/bpe.codes --vocabulary data1/data/bpe.vocab.{tgt} --vocabulary-threshold 50 < data1/data/dev/dev.{tgt}.txt > data1/data/dev/dev.{tgt}.bpe

!python subword-nmt/subword_nmt/apply_bpe.py -c data1/data/bpe.codes --vocabulary data1/data/bpe.vocab.{src} --vocabulary-threshold 50 < data1/data/test/test.{src}.txt > data1/data/test/test.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c data1/data/bpe.codes --vocabulary data1/data/bpe.vocab.{tgt} --vocabulary-threshold 50 < data1/data/test/test.{tgt}.txt > data1/data/test/test.{tgt}.bpe

In [40]:
!python -m sockeye.prepare_data -s data1/data/train/train.{src}.bpe -t data1/data/train/train.{tgt}.bpe --shared-vocab -o data1/{src}_{tgt}_{name}_data

[INFO:sockeye.utils] Sockeye: 3.1.34, commit 4c30942ddb523533bccb4d2cbb3e894e45b1db93, path /home/kvzhirnov/.conda/envs/diploma/lib/python3.11/site-packages/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1+cu117 (/home/kvzhirnov/.conda/envs/diploma/lib/python3.11/site-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /home/kvzhirnov/.conda/envs/diploma/lib/python3.11/site-packages/sockeye/prepare_data.py -s data1/data/train/train.chv.bpe -t data1/data/train/train.rus.bpe --shared-vocab -o data1/chv_rus_100K_Wiki_data
[INFO:sockeye.utils] Arguments: Namespace(config=None, source='data1/data/train/train.chv.bpe', source_factors=[], source_factors_use_source_vocab=[], target_factors=[], target_factors_use_target_vocab=[], target='data1/data/train/train.rus.bpe', end_of_prepending_tag=None, source_vocab=None, target_vocab=None, source_factor_vocabs=[], target_factor_vocabs=[], shared_vocab=True, num_words=(0, 0), word_min_count=(1, 1), pad_vocab_to_multiple_of=8, no_bucketi

In [41]:
!python -m sockeye.train -d data1/{src}_{tgt}_{name}_data -vs data1/data/dev/dev.{src}.bpe -vt data1/data/dev/dev.{tgt}.bpe --encoder transformer --decoder transformer --transformer-model-size 512 --transformer-feed-forward-num-hidden 2048 --transformer-dropout-prepost 0.1 --num-embed 512 --max-seq-len 100 --decode-and-evaluate 500 -o {src}_{tgt}_{name}_model --num-layers 6 --keep-last-params 3 --batch-size 2048 --optimized-metric bleu --max-num-checkpoint-not-improved 5 --amp

[INFO:sockeye.utils] Sockeye: 3.1.34, commit 4c30942ddb523533bccb4d2cbb3e894e45b1db93, path /home/kvzhirnov/.conda/envs/diploma/lib/python3.11/site-packages/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1+cu117 (/home/kvzhirnov/.conda/envs/diploma/lib/python3.11/site-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /home/kvzhirnov/.conda/envs/diploma/lib/python3.11/site-packages/sockeye/train.py -d data1/chv_rus_100K_Wiki_data -vs data1/data/dev/dev.chv.bpe -vt data1/data/dev/dev.rus.bpe --encoder transformer --decoder transformer --transformer-model-size 512 --transformer-feed-forward-num-hidden 2048 --transformer-dropout-prepost 0.1 --num-embed 512 --max-seq-len 100 --decode-and-evaluate 500 -o chv_rus_100K_Wiki_model --num-layers 6 --keep-last-params 3 --batch-size 2048 --optimized-metric bleu --max-num-checkpoint-not-improved 5 --amp
[INFO:sockeye.utils] Arguments: Namespace(config=None, source=None, source_factors=[], source_factors_use_source_vocab=[], target_fa

In [5]:
!python -m sockeye.translate -m {src}_{tgt}_{name}_model -i data1/data/test/test.{src}.bpe -o data1/data/test/test.{tgt}.pred.bpe --batch-size 128 --beam-size 3

[INFO:sockeye.utils] Sockeye: 3.1.34, commit 4c30942ddb523533bccb4d2cbb3e894e45b1db93, path /home/kvzhirnov/.conda/envs/diploma/lib/python3.11/site-packages/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1+cu117 (/home/kvzhirnov/.conda/envs/diploma/lib/python3.11/site-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /home/kvzhirnov/.conda/envs/diploma/lib/python3.11/site-packages/sockeye/translate.py -m chv_rus_100K_Wiki_model -i data1/data/test/test.chv.bpe -o data1/data/test/test.rus.pred.bpe --batch-size 128 --beam-size 3
[INFO:sockeye.utils] Arguments: Namespace(config=None, input='data1/data/test/test.chv.bpe', input_factors=None, json_input=False, output='data1/data/test/test.rus.pred.bpe', models=['chv_rus_100K_Wiki_model'], checkpoints=None, nbest_size=1, beam_size=3, greedy=False, beam_search_stop='all', batch_size=128, chunk_size=None, sample=None, seed=None, ensemble_mode='linear', bucket_width=10, max_input_length=None, max_output_length_num_stds=2, max_out

In [6]:
!sed -r 's/(@@ )|(@@ ?$)//g' data1/data/test/test.{tgt}.pred.bpe > data1/data/test/test.{tgt}.pred.txt

In [8]:
!sacrebleu data1/data/test/test.rus.txt -tok none -i data1/data/test/test.{tgt}.pred.txt

sacreBLEU: That's 100 lines that end in a tokenized period ('.')
sacreBLEU: It looks like you forgot to detokenize your test data, which may hurt your score.
sacreBLEU: If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
{
 "name": "BLEU",
 "score": 0.9,
 "signature": "nrefs:1|case:mixed|eff:no|tok:none|smooth:exp|version:2.3.1",
 "verbose_score": "12.8/1.3/0.3/0.1 (BP = 1.000 ratio = 1.007 hyp_len = 1096260 ref_len = 1088181)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "none",
 "smooth": "exp",
 "version": "2.3.1"
}


In [7]:
!python -m sockeye.translate --help

usage: translate.py [-h] [--config CONFIG] [--input INPUT]
                    [--input-factors INPUT_FACTORS [INPUT_FACTORS ...]]
                    [--json-input] [--output OUTPUT] --models MODELS
                    [MODELS ...] [--checkpoints CHECKPOINTS [CHECKPOINTS ...]]
                    [--nbest-size NBEST_SIZE] [--beam-size BEAM_SIZE]
                    [--greedy] [--beam-search-stop {all,first}]
                    [--batch-size BATCH_SIZE] [--chunk-size CHUNK_SIZE]
                    [--sample [SAMPLE]] [--seed SEED]
                    [--ensemble-mode {linear,log_linear}]
                    [--bucket-width BUCKET_WIDTH]
                    [--max-input-length MAX_INPUT_LENGTH]
                    [--max-output-length-num-stds MAX_OUTPUT_LENGTH_NUM_STDS]
                    [--max-output-length MAX_OUTPUT_LENGTH]
                    [--restrict-lexicon RESTRICT_LEXICON [RESTRICT_LEXICON ...]]
                    [--restrict-lexicon-topk RESTRICT_LEXICON_TOPK]
        